In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
# train.head()

In [ ]:
# test.head()

In [ ]:
# sub.head()

In [ ]:
# train_cols = []

# for data in train.columns:
#     if train[data].dtype == 'float':
#         # Scale 10
#         train_cols.append(
#             pd.DataFrame(columns=['{}_scale_10'.format(data)], 
#                          data=np.array(np.round(train[data] * 10), dtype='int')))

#         # Scale 100
#         train_cols.append(
#             pd.DataFrame(columns=['{}_scale_100'.format(data)], 
#                          data=np.array(np.round(train[data] * 100), dtype='int')))

In [ ]:
# # Concating data train
# train_data_scale = pd.concat(train_cols, axis=1)

In [ ]:
# # Train scale
# train_scale = pd.DataFrame(train_data_scale)

In [ ]:
# train_full = pd.merge(train, train_scale, left_index=True, right_index=True)

In [ ]:
# test_cols = []

# for data in test.columns:
#     if test[data].dtype == 'float':
#         # Scale 10
#         test_cols.append(
#             pd.DataFrame(columns=['{}_scale_10'.format(data)], 
#                          data=np.array(np.round(test[data] * 10), dtype='int')))

#         # Scale 100
#         test_cols.append(
#             pd.DataFrame(columns=['{}_scale_100'.format(data)], 
#                          data=np.array(np.round(test[data] * 100), dtype='int')))

In [ ]:
# # Concating data test
# test_data_scale = pd.concat(test_cols, axis=1)

In [ ]:
# # Train scale
# test_scale = pd.DataFrame(test_data_scale)

In [ ]:
# test_full = pd.merge(test, test_scale, left_index=True, right_index=True)

# **Divide Data**

In [ ]:
# X = train_full.drop(['id', 'loss'], axis=1)
# y = train_full['loss']

# testX = test_full.drop(['id'], axis=1)

In [ ]:
X = train.drop(['id', 'loss'], axis=1)
y = train['loss']

testX = test.drop(['id'], axis=1)

# Import Module

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
SEED = 99

np.random.seed(SEED)

In [ ]:
# Hyperparameter
# xgb_params = {
#     'n_estimators': 10000,
#     'learning_rate': 0.35,
#     'objective': 'reg:squarederror',
#     'subsample': 0.8031450486786944,
#     'colsample_bytree': 0.84,
#     'max_depth': 3,
#     'booster': 'gbtree', 
#     'reg_lambda': 35.1,
#     'reg_alpha': 14.68267919457715,
#     'random_state': 42,
#     'n_jobs': 4
# }

# xgb_params = {
#     'lambda': 1,
#     'alpha': 1,
#     'colsample_bytree': 0,
#     'subsample': .9,
#     'learning_rate': 0.17024722721525629,
#     'n_estimators': 9489,
#     'objective': 'reg:squarederror',
#     'max_depth': 3,
#     'gamma': 2,
#     'booster': 'gbtree',
#     'min_child_weight': 155,
#     'seed' : 38,
#     'random_state': 42,
#     'n_jobs': 4,
#     'sampling_method': 'uniform'
# }

# xgb_params = {
#     'lambda': 67.79737006663706,
#     'alpha': 40.12405005448161,
#     'colsample_bytree': 0.061613774851329205,
#     'subsample': 0.9556736521337416,
#     'learning_rate': 0.17024722721525629,
#     'n_estimators': 9489,
#     'max_depth': 3,
#     'booster': 'gbtree',
#     'min_child_weight': 155,
#     'seed' : 38
# }

# xgb_params = {
#     'lambda': 67.79737006663706,
#     'alpha': 40.12405005448161,
#     'colsample_bytree': 0.061613774851329205,
#     'subsample': 0.9556736521337416,
#     'learning_rate': 0.17024722721525629,
#     'n_estimators': 9489,
#     'objective': 'reg:squarederror',
#     'max_depth': 3,
#     'booster': 'gbtree',
#     'min_child_weight': 155,
#     'seed' : SEED,
#     'random_state': SEED,
#     'n_jobs': 4
# }

xgb_params = {'colsample_bytree': 0.31426530319123525,
            'gamma': 0.45519212869187453,
            'learning_rate': 0.06211498484648224,
            'max_depth': 6,
            'min_child_weight': 1.0,
            'n_estimators': 10000,
            'reg_alpha': 21.0,
            'reg_lambda': 0.9277740406539857,
            'subsample': 0.6254798229473668,
            'booster': 'gbtree', 
            'random_state': SEED,
            'seed': 91,
            'n_jobs': -1}

In [ ]:
#Setting the kfold parameters
kf = KFold(n_splits = 7, shuffle = True, random_state = SEED)

oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
mean_rmse = 0

for num, (train_id, valid_id) in enumerate(kf.split(X)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    testX = scaler.transform(testX)
    X_valid = scaler.transform(X_valid)
    
    model = XGBRegressor(**xgb_params, tree_method='gpu_hist')
    model.fit(X_train, y_train,
             verbose = False,
             eval_set = [(X_train, y_train), (X_valid, y_valid)],
             eval_metric = "rmse",
             early_stopping_rounds = 100)
    
    #Mean of the predictions
    preds += model.predict(testX) / 7 # Splits
    
    #Mean of feature importance
    model_fi += model.feature_importances_ / 7 #splits
    
    #Out of Fold predictions
    oof_preds[valid_id] = model.predict(X_valid)
    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof_preds[valid_id]))
    print(f"Fold {num} | RMSE: {fold_rmse}")
    
    mean_rmse += fold_rmse / 7
    
print(f"\nOverall RMSE: {mean_rmse}")

In [ ]:
sub.loss = preds
sub.head()

In [ ]:
sub.to_csv('submission_with_xgbregressor5.csv', index=False)
print('sending')